In [4]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from pprint import pprint
import spacy
import pickle
import re
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
import glob
import pandas as pd

path = r'C:\Users\ivank\Desktop\Scraper\Hearings\2024'
csv_files = glob.glob(path + '\*.csv')

df_list = (pd.read_csv(file) for file in csv_files)

df = pd.concat(df_list,ignore_index=True)

<>:5: DeprecationWarning: invalid escape sequence '\*'
<>:5: DeprecationWarning: invalid escape sequence '\*'
C:\Users\ivank\AppData\Local\Temp\ipykernel_26028\2555414059.py:5: DeprecationWarning: invalid escape sequence '\*'
  csv_files = glob.glob(path + '\*.csv')


In [6]:
bulgarian_stopwords = [
    "а", "автентичен", "аз", "ако", "ала", "бе", "без", "беше", "би", "бивш",
    "бивша", "бившо", "бил", "била", "били", "било", "благодаря", "близо",
    "бъдат", "бъде", "бяха", "в", "вас", "ваш", "ваша", "вероятно", "вече",
    "взема", "ви", "вие", "винаги", "внимава", "време", "все", "всеки",
    "всички", "всичко", "всяка", "във", "въпреки", "върху", "г", "ги",
    "главен", "главна", "главно", "глас", "го", "година", "години",
    "годишен", "д", "да", "дали", "два", "двама", "двамата", "две",
    "двете", "ден", "днес", "дни", "до", "добра", "добре", "добро",
    "добър", "докато", "докога", "дори", "досега", "доста", "друг",
    "друга", "други", "е", "евтин", "едва", "един", "една", "еднаква",
    "еднакви", "еднакъв", "едно", "екип", "ето", "живот", "за", "забавям",
    "зад", "заедно", "заради", "засега", "заспал", "затова", "защо",
    "защото", "и", "из", "или", "им", "има", "имат", "иска", "й",
    "каза", "как", "каква", "какво", "както", "какъв", "като", "кога",
    "когато", "което", "които", "кой", "който", "колко", "която",
    "къде", "където", "към", "лесен", "лесно", "ли", "лош", "м", "май",
    "малко", "ме", "между", "мек", "мен", "месец", "ми", "много",
    "мнозина", "мога", "могат", "може", "мокър", "моля", "момента", "му",
    "н", "на", "над", "назад", "най", "направи", "напред", "например",
    "нас", "не", "него", "нещо", "нея", "ни", "ние", "никой", "нито",
    "нищо", "но", "нов", "нова", "нови", "новина", "някои", "някой",
    "няколко", "няма", "обаче", "около", "освен", "особено", "от",
    "отгоре", "отново", "още", "пак", "по", "повече", "повечето", "под",
    "поне", "поради", "после", "почти", "прави", "пред", "преди",
    "през", "при", "пък", "първата", "първи", "първо", "пъти", "равен",
    "равна", "с", "са", "сам", "само", "се", "сега", "си", "син",
    "скоро", "след", "следващ", "сме", "смях", "според", "сред",
    "срещу", "сте", "съм", "със", "също", "т", "т.н.", "тази", "така",
    "такива", "такъв", "там", "твой", "те", "тези", "ти", "то", "това",
    "тогава", "този", "той", "толкова", "точно", "три", "трябва", "тук",
    "тъй", "тя", "тях", "у", "утре", "харесва", "хиляди", "ч", "часа",
    "че", "често", "чрез", "ще", "щом", "юмрук", "я", "як"
]


In [9]:
revival_df = df[df['Партия'] == 'ВЪЗРАЖДАНЕ']
revival_statements = revival_df['Изказване']

tokenized_statements = [doc.split() for doc in revival_statements]

# Step 1: Use CountVectorizer


vectorizer = CountVectorizer(stop_words=bulgarian_stopwords)
X = vectorizer.fit_transform(revival_statements)  # This gives a sparse matrix

# Step 2: Convert to Gensim Dictionary and Corpus
id2word = corpora.Dictionary(tokenized_statements)
corpus = [id2word.doc2bow(text) for text in tokenized_statements]

# Step 3: Train LDA Model
lda_model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=6,
    random_state=0,
    chunksize=10000,
    alpha='auto',
    per_word_topics=False
)

doc_lda = lda_model[corpus]


# Generate pyLDAvis visualization
lda_display = gensimvis.prepare(lda_model, corpus, id2word)
pyLDAvis.display(lda_display)